# First Exercise

Pandas seems mandatory to deal with such huge datasets. Therefore we are going to use it. Once loaded with read_csv we know the number of rows with the details of the dataframe in the Variable Explorer but once's can also calculate the number of rows like this  

In [ ]:
from pandas import read_csv
from GeoBases import GeoBase

bookings_location = r'/Volumes/RemiSDCard/Amadeus/bookings.csv'

searches_location = r'/Volumes/RemiSDCard/Amadeus/searches.csv'

def csv_line_counter(filename):
    with open(filename) as f:
        for i,l in enumerate(f):
            pass
        f.close()
    return i + 1

row_count_bookings = csv_line_counter(bookings_location)
row_count_searches = csv_line_counter(searches_location)

print "bookings.csv has %s rows, header included" % row_count_bookings
print "searches.csv has %s rows, header included" % row_count_searches

bookings.csv has 10000011 rows, header included 

searches.csv has 20390199 rows, header included

Noticed that the function is considering the header as a row. 

# Second Exercise

The goal of this exercise if to find the top 10 arrival airports in the world in 2013, using the bookings file. Since we only need the columns arr_port and pax, thus we are going to use pandas useful capaibilities to only import the two interesting columns and save space and time. 

We will use GeoBases https://github.com/opentraveldata/geobases to get the name of the airports concerned.